In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

In [2]:
from nlp_pipeline import *

In [3]:
oof21 = pd.read_csv("submissions\\oof_train21.csv")
sub21 = pd.read_csv("submissions\\submission21.csv")
oof22 = pd.read_csv("submissions\\oof_train22.csv")
sub22 = pd.read_csv("submissions\\submission22.csv")
ooflstm4 = pd.read_csv("submissions\\lstm_ft_oof_template.csv")
sublstm4 = pd.read_csv("submissions\\lstm4.csv")
oof29 = pd.read_csv("submissions\\oof_train29.csv")
sub29 = pd.read_csv("submissions\\submission29.csv")
oof31 = pd.read_csv("submissions\\oof_train31.csv")
sub31 = pd.read_csv("submissions\\submission31.csv")

In [4]:
train = pd.read_csv('data\\train.csv').fillna(' ')
test = pd.read_csv('data\\test.csv').fillna(' ')

In [5]:
labels = [c for c in oof21.columns][1:8]

In [6]:
pretrained = "data\\crawl-300d-2M.vec"
feature_funcs = [len, asterix_freq, uppercase_freq, line_change_freq, rep_freq, question_freq]
transforms = [tokenize]
logreg = LogisticRegression(C=0.2, class_weight='balanced', solver='newton-cg', max_iter=10)
logreg.name = "Logistic regression stacker"
gbm = lgb.LGBMClassifier(metric="auc", num_leaves=31, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.9, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.5)
gbm.name = "LightGBM stacker"
models = [gbm]

In [7]:
pipe = NlpPipeline(train, test, "comment_text", labels, feature_funcs, transforms, models, word_index=None, pretrained=pretrained)

In [8]:
pipe.engineer_features()

Engineering features


In [9]:
train_meta = np.hstack([np.array(oof21[labels]), np.array(oof22[labels]), np.array(ooflstm4[labels]), np.array(oof29[labels]), np.array(oof31[labels])])
test_meta = np.hstack([np.array(sub21[labels]), np.array(sub22[labels]), np.array(sublstm4[labels]), np.array(sub29[labels]), np.array(sub31[labels])])

In [10]:
pipe.train_features = np.hstack([pipe.train_features, train_meta])
pipe.test_features = np.hstack([pipe.test_features, test_meta])

In [17]:
pipe.cross_val()

Cross-validating
LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        learning_rate=0.1, max_depth=-1, metric='auc',
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.5, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=1)
Cross-validating toxic
roc_auc: 0.9851925414892249
Cross-validating severe_toxic
roc_auc: 0.9903692335089094
Cross-validating obscene
roc_auc: 0.9938424809553432
Cross-validating threat
roc_auc: 0.9924120435441581
Cross-validating insult
roc_auc: 0.9876524159715508
Cross-validating identity_hate
roc_auc: 0.988557206382638


In [13]:
pipe.cv_scores

{'LightGBM stacker': 0.9898947670691306}

In [14]:
pipe.fit_predict()

Fitting and predicting
Fitting submission classifier for toxic
Fitting submission classifier for severe_toxic
Fitting submission classifier for obscene
Fitting submission classifier for threat
Fitting submission classifier for insult
Fitting submission classifier for identity_hate


In [15]:
pipe.create_submission(oof=False)

Creating submissions


In [ ]:
pipe

In [ ]:
from scipy.special import expit, logit

In [ ]:
def rescale(x):
    return expit(logit(x) - 0.5)

In [ ]:
rescale(0.5)

In [ ]:
train.toxic.mean()